# HARDWARE

## Setup

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

"""
To change attributes:
1) Change ATTRIBUTE and you're good to go
"""
ATTRIBUTE = 'polarity'
COUNTER = '_new'
PARALLEL = 1
PARALLEL_F = 1
PARALLEL_EXTRACTION = 1
TRAIN_SIZE = 5
DEV_SIZE = 5
TEST_SIZE = 5

import os
import sys

os.environ['SNORKELDBNAME'] = ATTRIBUTE + str(COUNTER) +'2'
os.environ['SNORKELDB'] = 'postgres://localhost:5432'# + os.environ['SNORKELDBNAME']
        
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/fonduer/tables/')
snorkel_postgres = os.environ['SNORKELDB'].startswith('postgres')
print snorkel_postgres

/Users/sen/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1085: UserWarning: Duplicate key in file "/Users/sen/.matplotlib/matplotlibrc", line #467
  (fname, cnt))


True


In [2]:
import os
if snorkel_postgres:
#     os.environ['SNORKELDBNAME'] = ATTRIBUTE + str(COUNTER) +'2'
    print os.system("dropdb " + os.environ['SNORKELDBNAME'])
    print os.system("createdb " + os.environ['SNORKELDBNAME'])
    print "SNORKELDB = %s" % os.environ['SNORKELDB']
    print "SNORKELDBNAME = %s" % os.environ['SNORKELDBNAME']
else:
    try:
        os.remove('snorkel.db')
    except:
        pass

from fonduer import SnorkelSession
session = SnorkelSession()

0
0
SNORKELDB = postgres://localhost:5432
SNORKELDBNAME = polarity_new2


## Parsing

In [3]:
# from fonduer.async_parser import HTMLParser, AsyncOmniParser
from fonduer.parser import HTMLPreprocessor, OmniParser
from fonduer.models import Document, Sentence
print "Starting async parse..."


# docs_path = os.environ['SNORKELHOME'] + '/tutorials/fonduer/tables/data/hardware/train_digikey/html/'
# pdf_path = os.environ['SNORKELHOME'] + '/tutorials/fonduer/tables/data/hardware/train_digikey/pdf/'
docs_path = os.environ['SNORKELHOME'] + '/tutorials/fonduer/tables/data/hardware/dev/html/'
pdf_path = os.environ['SNORKELHOME'] + '/tutorials/fonduer/tables/data/hardware/dev/pdf/'

doc_preprocessor = HTMLPreprocessor(docs_path, max_docs=20)

# corpus_parser = OmniParser(blacklist=['style'], flatten=['span','br'], tabular=True, lingual=True, visual=True, pdf_path=pdf_path)
corpus_parser = OmniParser(structural=True, lingual=True, visual=True, blacklist=['style'], flatten=['span','br'], pdf_path=pdf_path)
corpus_parser.apply(doc_preprocessor, parallelism=10)

# corpus_parser = AsyncOmniParser(blacklist=['style'], flatten=['span','br'], 
#                                  tabular=True, lingual=True,
#                                  visual=True)

# %time corpus_parser.apply(doc_preprocessor, docs_path, pdf_path, session, \
#                             max_docs=TRAIN_SIZE, parallel=PARALLEL)
# session.commit()
print "Documents:", session.query(Document).count()
print "Sentences:", session.query(Sentence).count()

Starting async parse...
Clearing existing...
Running UDF...
Documents: 20
Sentences: 0


In [4]:
docs = session.query(Document).order_by(Document.name).all()

In [5]:
for i, d in enumerate(docs):
    print i, len(d.phrases), len(d.tables)

0 799 9
1 662 9
2 784 14
3 662 11
4 512 11
5 700 10
6 528 10
7 158 2
8 228 7
9 511 10
10 331 6
11 528 9
12 432 5
13 567 8
14 340 6
15 432 5
16 416 9
17 820 9
18 202 2
19 301 4


In [6]:
for i, d in enumerate(docs):
    print d.phrases[0].text, d.phrases[0].top, d.phrases[0].bottom

BC546 [54] [71]
2N3906 MMBT3906 PZT3906 - PNP General-Purpose Amplifier [54, 54, 54, 54, 54, 54, 54] [427, 427, 427, 427, 427, 427, 427]
2N3906 - General Purpose Transistors, PNP Silicon [54, 193, 92, 92, 112, 193, 136, 136] [71, 209, 110, 110, 129, 209, 153, 153]
2n4123. [207, 188] [217, 203]
2N4124/MMBT4124 NPN General Purpose Amplifier [54, 296, 296, 296, 296] [195, 309, 309, 309, 309]
2N6426 [54] [71]
Symbol [382] [392]
AUKCS04635-1 [649] [663]
BC182 NPN General Purpose Amplifier [96, 192, 192, 192, 192] [140, 203, 203, 203, 203]
BC182 [54] [71]
BC337 BC338 - NPN Epitaxial Silicon Transistor [61, 61, 61, 61, 61, 61, 61] [341, 341, 341, 341, 341, 341, 341]
BC337 - NPN Amplifier Transistors [50, 305, 136, 335, 385] [73, 314, 153, 346, 396]
BC546, BC547, BC548, BC549, BC550 - NPN Epitaxial Silicon Transistor [61, 255, 147, 255, 147, 255, 268, 255, 147, 147, 61, 61, 61, 61] [480, 266, 164, 266, 164, 266, 277, 266, 164, 164, 480, 480, 480, 480]
BC546-BC548C(TO-92). [233, 293, 312, 293, 

In [7]:
from fonduer.models import Document, Phrase


session.query(Phrase).count()

9913L

### Save Corpus

In [8]:
# If necessary
if not snorkel_postgres:
    import os
    os.system('cp snorkel.db snorkel.db\ corpus');

## Extraction

In [9]:
# # If necessary:
# import os
# os.remove('snorkel.db');
# os.system('cp snorkel.db\ corpus snorkel.db');

# import sys
# sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

# from snorkel import SnorkelSession
# session = SnorkelSession()

In [10]:
from fonduer.models import candidate_subclass

Part_Attr = candidate_subclass('Part_Attr', ['part','attr'])

In [11]:
# session.rollback()

### Define Matchers

In [12]:
from hardware_matchers import get_matcher

dict_path = os.environ['SNORKELHOME'] +\
    '/tutorials/fonduer/tables/data/hardware/gold_raw/digikey_part_dictionary.csv'
part_matcher = get_matcher('part', dict_path)
attr_matcher = get_matcher(ATTRIBUTE)

Using combined matcher.


### Define ContextSpaces

In [13]:
from hardware_spaces import get_space
    
part_ngrams = get_space('part')
attr_ngrams = get_space(ATTRIBUTE)

### Define Candidate Throttler

In [14]:
from hardware_throttlers import get_throttler

throttler = get_throttler(ATTRIBUTE)
# throttler = None
print throttler

<function polarity_throttler at 0x114325938>


### Run CandidateExtractor

In [15]:
from fonduer.candidates import CandidateExtractor
# from snorkel.utils import get_ORM_instance
# from fonduer.async_candidates import parallel_extract

candidate_extractor = CandidateExtractor(Part_Attr, 
                        [part_ngrams, attr_ngrams], 
                        [part_matcher, attr_matcher], 
                        throttler=throttler)

%time candidate_extractor.apply(docs, split=0)

# corpus_names = ['Hardware Train', 'Hardware Dev']
# if TEST_SIZE:
#     corpus_names.append('Hardware Test')
# for corpus_name in corpus_names:
#     corpus = get_ORM_instance(Corpus, session, corpus_name)
#     print "Extracting Candidates from %s" % corpus
#     %time candidates = parallel_extract(session, candidate_extractor, corpus, \
#                                         corpus_name + ' Candidates', \
#                                         parallel=PARALLEL_EXTRACTION)
#     session.add(candidates)
#     print "%s contains %d Candidates" % (candidates, len(candidates))
# session.commit()

Clearing existing...
Running UDF...
[========================================] 100%
CPU times: user 30 s, sys: 722 ms, total: 30.7 s
Wall time: 39.2 s


In [16]:
train_cands = session.query(Part_Attr).filter(Part_Attr.split == 0).all()
print "Number of candidates:", len(train_cands)

Number of candidates: 3045


In [17]:
print train_cands[0]
print train_cands[0][0].sentence.text
print train_cands[0][1].sentence.text

Part_Attr(Span("BC546", sentence=4941, chars=[86,90], words=[11,11]), Span("NPN", sentence=445, chars=[0,2], words=[0,0]))
(IC= 2.0 mA, VCE= 5.0 V)                                                              BC546
NPN Silicon


### Second Pass

In [18]:
# from hardware_utils import get_gold_parts_by_doc, get_manual_parts_by_doc
# from snorkel.utils import get_ORM_instance
# from snorkel.models import Corpus

# corpus = get_ORM_instance(Corpus, session, 'Hardware Dev')

# # parts_by_doc = get_gold_parts_by_doc()
# parts_by_doc = get_manual_parts_by_doc(corpus.documents.all())
# # parts_by_doc = None

In [19]:
# import cPickle as pickle
# pickle_file = os.environ['SNORKELHOME'] + '/tutorials/tables/sandbox/parts_by_doc_dev.pkl'

# with open(pickle_file, 'w') as f:
#     pickle.dump(parts_by_doc, f)

In [20]:
import cPickle as pickle
pickle_file = os.environ['SNORKELHOME'] + '/tutorials/fonduer/tables/sandbox/parts_by_doc_dev.pkl'
with open(pickle_file, 'r') as f:
    parts_by_doc = pickle.load(f)

In [21]:
for i, d in enumerate(docs):
    print d.name

112823
2N3906
2N3906-D
2N4123-D
2N4124
2N6426-D
2N6427
AUKCS04635-1
BC182
BC182-D
BC337
BC337-D
BC546
BC546-BC548C(TO-92)
BC546A_Series_B14-521026
BC547
BC818-40LT1-D
bc546-d
bc546_diotec
bc550


### Assess Recall

In [22]:
# from fonduer.models import Corpus, CandidateSet
from hardware_utils import entity_level_f1

# corpus = get_ORM_instance(Corpus, session, 'Hardware Dev')
# candidates = get_ORM_instance(CandidateSet, session, 'Hardware Dev Candidates')
candidates = train_cands

gold_file = os.environ['SNORKELHOME'] + \
    '/tutorials/fonduer/tables/data/hardware/dev/hardware_dev_gold.csv'
%time (ctp, cfp, cfn) = entity_level_f1(candidates, gold_file, ATTRIBUTE, docs, parts_by_doc)

Preparing candidates...
[========================================] 100%
Scoring on Entity-Level Gold Data
Corpus Precision 0.908
Corpus Recall    1.0
Corpus F1        0.952
----------------------------------------
TP: 258 | FP: 26 | FN: 0

CPU times: user 3.15 s, sys: 215 ms, total: 3.36 s
Wall time: 3.73 s


## Gold Labels

In [23]:
# If necessary
# import os
# os.remove('snorkel.db');
# os.system('cp snorkel.db\ candidates snorkel.db');

# import sys
# sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

# from snorkel import SnorkelSession
# session = SnorkelSession()

# from snorkel.models import candidate_subclass
# Part_Attr = candidate_subclass('Part_Attr', ['part','attr'])

In [24]:
# import os
# from fonduer.models import CandidateSet
# from hardware_utils import load_hardware_labels

# data_sets = ['Dev']
# gold_file = {}
# gold_file['Dev'] = os.environ['SNORKELHOME'] + '/tutorials/fonduer/tables/data/hardware/dev/hardware_dev_gold.csv'
# if TEST_SIZE:
#     data_sets.append('Test')
#     gold_file['Test'] = os.environ['SNORKELHOME'] + '/tutorials/fonduer/tables/data/hardware/test/hardware_test_gold.csv'
# for data_set in data_sets:
#     candidate_set_name = 'Hardware %s Candidates' % data_set
#     candidates = session.query(CandidateSet).filter(
#         CandidateSet.name == candidate_set_name).one()
#     label_set_name = 'Hardware %s Candidates -- Gold' % data_set
#     annotation_key_name = 'Hardware %s Labels -- Gold' % data_set
#     %time gold_candidates, annotation_key = load_hardware_labels(session,\
#                            label_set_name, \
#                            annotation_key_name, \
#                            candidates, \
#                            gold_file[data_set], \
#                            ATTRIBUTE)
#     candidates_gold = session.query(CandidateSet).filter(
#         CandidateSet.name == candidate_set_name + ' -- Gold').one()
#     print "%d/%d Candidates in %s have positive Labels" % (
#         len(candidates_gold), len(candidates), candidates)

## Features

In [28]:
from fonduer.features.features import get_all_feats
for f in get_all_feats(candidates[0]):
    print f

('CORE_e1_SPAN_TYPE_[EXPLICIT]', 1)
('CORE_e1_STARTS_WITH_CAPITAL', 1)
('CORE_e1_LENGTH_1', 1)
('CORE_e2_SPAN_TYPE_[EXPLICIT]', 1)
('CORE_e2_STARTS_WITH_CAPITAL', 1)
('CORE_e2_LENGTH_1', 1)
(u'DDL_e1_WORD_SEQ_[BC546]', 1)
(u'DDL_e1_LEMMA_SEQ_[bc546]', 1)
(u'DDL_e1_POS_SEQ_[NN]', 1)
(u'DDL_e1_DEP_SEQ_[dep]', 1)
(u'DDL_e1_W_LEFT_1_[-rrb-]', 1)
(u'DDL_e1_W_LEFT_POS_1_[-RRB-]', 1)
(u'DDL_e1_W_LEFT_2_[v -rrb-]', 1)
(u'DDL_e1_W_LEFT_POS_2_[NN -RRB-]', 1)
(u'DDL_e1_W_LEFT_3_[_NUMBER v -rrb-]', 1)
(u'DDL_e1_W_LEFT_POS_3_[CD NN -RRB-]', 1)
(u'DDL_e2_WORD_SEQ_[NPN]', 1)
(u'DDL_e2_LEMMA_SEQ_[NPN]', 1)
(u'DDL_e2_POS_SEQ_[NNP]', 1)
(u'DDL_e2_DEP_SEQ_[compound]', 1)
(u'DDL_e2_W_LEFT_1_[Silicon]', 1)
(u'DDL_e2_W_LEFT_POS_1_[NNP]', 1)
(u'DDL_e2_W_LEFT_2_[NPN Silicon]', 1)
(u'DDL_e2_W_LEFT_POS_2_[NNP NNP]', 1)
(u'DDL_e2_W_RIGHT_1_[Silicon]', 1)
(u'DDL_e2_W_RIGHT_POS_1_[NNP]', 1)
(u'DDL_e2_W_LEMMA_L_1_R_1_[Silicon]_[Silicon]', 1)
(u'DDL_e2_W_POS_L_1_R_1_[NNP]_[NNP]', 1)
(u'DDL_e2_W_LEMMA_L_2_R_1_[NPN Si

### Extract Features

In [ ]:
from fonduer.models import CandidateSet
from snorkel.utils import get_ORM_instance

train = get_ORM_instance(CandidateSet, session, 'Hardware Train Candidates')
dev   = get_ORM_instance(CandidateSet, session, 'Hardware Dev Candidates')
test  = get_ORM_instance(CandidateSet, session, 'Hardware Test Candidates')

if snorkel_postgres:
    from fonduer.async_annotations import annotate
    print "Starting async featurization..."
    %time F_train = annotate(train, parallel=PARALLEL_F, dynamic_scheduling=False)
    %time F_dev   = annotate(dev, parallel=PARALLEL_F, dynamic_scheduling=False,\
                             keyset = 'Hardware Train Candidates')
    if TEST_SIZE:
        %time F_test = annotate(test, parallel=PARALLEL_F, dynamic_scheduling=False,\
                                keyset = 'Hardware Train Candidates')
    
else:
    from fonduer.models import CandidateSet
    from fonduer.fast_annotations import FeatureManager
    from snorkel.utils import get_ORM_instance

    print "Starting sync featurization..."
    feature_manager = FeatureManager()
    %time F_train = feature_manager.create(session, train, 'Train Features')
    %time F_dev = feature_manager.update(session, dev, 'Train Features', expand_key_set=False)

In [ ]:
# If necessary:
if not snorkel_postgres:
    import os
    os.system('cp snorkel.db snorkel.db\ featurized');

## LFs

In [ ]:
# If necessary
# import os
# os.remove('snorkel.db');
# os.system('cp snorkel.db\ featurized snorkel.db');

# from snorkel import SnorkelSession
# session = SnorkelSession()

# import sys
# sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

# from snorkel.models import candidate_subclass
# Part_Attr = candidate_subclass('Part_Attr', ['part','attr'])

# from snorkel.models import CandidateSet
# train = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Train Candidates').one()
# dev = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Dev Candidates').one()

# from snorkel.annotations import FeatureManager, LabelManager
# feature_manager = FeatureManager()
# %time F_train = feature_manager.load(session, train, 'Train Features')
# %time F_dev = feature_manager.load(session, dev, 'Train Features')

### Define LFs

In [29]:
from hardware_lfs import get_lfs

LFs = get_lfs(ATTRIBUTE)

In [30]:
for lf in LFs:
    print lf(candidates[0])

0
0
0
0
1
0
0
0
0
1
1
0


### Apply LFs

In [ ]:
print train.name
print len(train)

In [ ]:
if snorkel_postgres:
    from fonduer.async_annotations import annotate
    %time L_train = annotate(train, parallel=PARALLEL_F, update=False, lfs=LFs, dynamic_scheduling=False, storage='COO')
else:
    from fonduer.fast_annotations import LabelManager
    label_manager = LabelManager()
    %time L_train = label_manager.create(session, train, 'LF Labels', f=LFs)
L_train

### Assess LF accuracy

In [ ]:
%time L_train.lf_stats()

In [ ]:
# If necessary
if not snorkel_postgres:
    import os
    os.system('cp snorkel.db snorkel.db\ features');

## Learn and Evaluate

In [ ]:
# If necessary:
# import os
# os.remove('snorkel.db');
# os.system('cp snorkel.db\ features snorkel.db');

# from snorkel import SnorkelSession
# session = SnorkelSession()

# import sys
# sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

# from snorkel.models import candidate_subclass
# Part_Attr = candidate_subclass('Part_Attr', ['part','attr'])

# from snorkel.models import CandidateSet
# train = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Training Candidates').one()
# dev = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Development Candidates').one()

# from snorkel.annotations import FeatureManager, LabelManager
# feature_manager = FeatureManager()
# %time F_train = feature_manager.load(session, train, 'Train Features')
# %time F_dev = feature_manager.load(session, dev, 'Train Features')

# label_manager = LabelManager()
# %time L_train = label_manager.load(session, train, 'LF Labels')

In [ ]:
from snorkel.learning import GenerativeModel

gen_model = GenerativeModel()
%time gen_model.train(L_train, epochs=500, decay=0.95, step_size=0.1/L_train.shape[0], reg_param=1e-6)

In [ ]:
train_marginals = gen_model.marginals(L_train)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(train_marginals, bins=20)
plt.show()

In [ ]:
gen_model.weights.lf_accuracy()

In [ ]:
# from snorkel.learning import NaiveBayes

# gen_model = NaiveBayes()
# %time gen_model.train(L_train, n_iter=2000, rate=1e-3, mu=1e-6)
# train_marginals = gen_model.marginals(L_train)

In [ ]:
pprint(zip([lf.__name__ for lf in LFs], gen_model.weights.lf_accuracy_log_odds))
print min(train_marginals)
print max(train_marginals)

In [ ]:
from snorkel.learning import SparseLogisticRegression

disc_model = SparseLogisticRegression()
%time disc_model.train(F_train, train_marginals, n_epochs=200, lr=0.001)

In [ ]:
# from snorkel.learning import LogReg

# disc_model = LogReg()
# %time disc_model.train(F_train, train_marginals, n_iter=10000, rate=1e-3)

In [ ]:
dev_gold = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Dev Candidates -- Gold').one()

In [ ]:
from fonduer.annotations import LabelManager
label_manager = LabelManager()
L_dev = label_manager.load(session, dev, 'Hardware Dev Labels -- Gold')
L_dev.shape

In [ ]:
tp, fp, tn, fn = disc_model.score(session, F_dev, L_dev)

In [ ]:
from fonduer.models import Corpus
from hardware_utils import entity_level_f1
import os

gold_file = os.environ['SNORKELHOME'] + '/tutorials/fonduer/tables/data/hardware/dev/hardware_dev_gold.csv'
corpus = session.query(Corpus).filter(Corpus.name == 'Hardware Dev').one()
%time (TP, FP, FN) = entity_level_f1(tp.union(fp), gold_file, ATTRIBUTE, corpus, parts_by_doc=None)
%time (TP, FP, FN) = entity_level_f1(tp.union(fp), gold_file, ATTRIBUTE, corpus, parts_by_doc=parts_by_doc)

### Error Analysis

### Oracle

In [ ]:
from hardware_utils import get_gold_parts_by_doc, get_manual_parts_by_doc
from snorkel.utils import get_ORM_instance
from fonduer.models import Corpus

parts_by_doc = get_gold_parts_by_doc()
(TP, FP, FN) = entity_level_f1(tp.union(fp), gold_file, ATTRIBUTE, corpus, parts_by_doc)

### Test Results

In [ ]:
if TEST_SIZE:
    from fonduer.annotations import LabelManager
    label_manager = LabelManager()
    L_test = label_manager.load(session, test, 'Hardware Test Labels -- Gold')
    L_test.shape
    
    tp, fp, tn, fn = disc_model.score(session, F_test, L_test, b=0.91)
    
    from hardware_utils import get_gold_parts_by_doc, get_manual_parts_by_doc
    from snorkel.utils import get_ORM_instance
    from fonduer.models import Corpus

    corpus = get_ORM_instance(Corpus, session, 'Hardware Test')

    # parts_by_doc_test = get_manual_parts_by_doc(corpus.documents.all())
    # parts_by_doc_test = None
    import cPickle as pickle
    pickle_file = os.environ['SNORKELHOME'] + '/tutorials/fonduer/tables/sandbox/parts_by_doc_test.pkl'
    with open(pickle_file, 'r') as f:
        parts_by_doc_test = pickle.load(f)

    from hardware_utils import entity_level_f1

    gold_file = os.environ['SNORKELHOME'] + '/tutorials/fonduer/tables/data/hardware/test/hardware_test_gold.csv'
    (TP, FP, FN) = entity_level_f1(tp.union(fp), gold_file, ATTRIBUTE, corpus)
    (TP, FP, FN) = entity_level_f1(tp.union(fp), gold_file, ATTRIBUTE, corpus, parts_by_doc_test)